In [1]:
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import os
import json

In [ ]:
def make_save_dir(save_dir):
    if save_dir != None:
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            
def make_subset(annotations, cls_div_ids):
    dict_annos = defaultdict(list)
    for id, anno_details in tqdm(annotations.items()):
        subset_annos = []
        for anno in anno_details:
            if anno["LabelName"] in cls_div_ids:
                subset_annos.append(anno)
        if len(subset_annos)>0:
            dict_annos[id] = subset_annos

    idx_to_ids = {i:key for i, (key, _) in enumerate(dict_annos.items())}
    return dict_annos, idx_to_ids

def make_img_id_subset(annotations, cls_div_ids, valid_img_ids):
    dict_annos = defaultdict(list)
    
    for id in valid_img_ids:
        try:
            anno_details = annotations[id]
        except:
            print(id)
        subset_annos = []
        for anno in anno_details:
            if anno["LabelName"] in cls_div_ids:
                subset_annos.append(anno)
        if len(subset_annos)>0:
            dict_annos[id] = subset_annos

    idx_to_ids = {i:key for i, (key, _) in enumerate(dict_annos.items())}
    return dict_annos, idx_to_ids

def get_class_levels(hier):
    level1=[]
    level2=[]
    level3=[]
    level4=[]
    level5=[]

    for l2 in hier['Subcategory']:
        level5.append(l2['LabelName'])
        try:
            for j in l2['Subcategory']:
                level4.append(j['LabelName'])
                try:
                    for k in j['Subcategory']:
                        level3.append(k['LabelName'])
                        try:
                            for a in k['Subcategory']:
                                level2.append(a['LabelName'])
                                try:
                                    for b in a['Subcategory']:
                                        level1.append(b['LabelName'])
                                except:
                                    pass
                        except:
                            pass
                except:
                    pass
        except:
            pass
    level1 = list(set(level1))
    level2 = list(set(level2))
    level3 = list(set(level3))
    level4 = list(set(level4))       
    level5 = list(set(level5))   
    return [level1,level2,level3,level4,level5]

def get_required_human_list(human_labels_list, cls_to_names):
    important_human_label_list = []
    for hl in human_labels_list:
        if hl in cls_to_names:
            important_human_label_list.append(hl)
    return important_human_label_list

In [ ]:
hierarchy_dir = '../dataset/challenge-2019-label500-hierarchy.json'
clsids_to_names_dir = "../data_info/all/clsids_to_names.json"
train_anno_dir = "../data_info/all/train/annotations/train-anno.json"
valid_anno_dir = "../data_info/all/valid/annotations/valid-anno.json"
human_labels_dir = "../dataset/class-ids-human-body-parts-and-mammal.txt"
save_dir = "../data_info/subsets"

In [ ]:
clsids_to_names = json.load(open(clsids_to_names_dir,'r'))
hier = json.load(open(hierarchy_dir,'r'))
train_anno = json.load(open(train_anno_dir,'r'))
valid_anno = json.load(open(valid_anno_dir,'r'))
human_labels_list = list(pd.read_csv(human_labels_dir, header=None)[0])

In [ ]:
human_labels_list = get_required_human_list(human_labels_list, clsids_to_names)
list_of_divisions = get_class_levels(hier)

In [ ]:
# remove human labels from other class divisions
filter_list_of_dvisions = []
for divisions in list_of_divisions:
    temp = [x for x in divisions if x not in human_labels_list]
    filter_list_of_dvisions.append(temp)

In [ ]:
all_divisions = filter_list_of_dvisions+ [human_labels_list]

In [ ]:
for div in all_divisions:
    print(len(div))

In [ ]:
def create_multiple_subsets(anno, all_divisions, save_dir, typ, clsids_to_names):
    for i, div in enumerate(all_divisions):
        dict_annos, idx_to_ids = make_subset(anno, div)
        print(i,":",len(dict_annos), len(idx_to_ids))
        curr_save_dir = save_dir+"/"+ str(i)
        make_save_dir(curr_save_dir+"/annotations")
        with open(curr_save_dir +"/annotations"+"/"+typ+"-anno.json", 'w') as fp:
            json.dump(dict(dict_annos), fp)

        with open(curr_save_dir+"/annotations"+"/"+typ+"-idx_to_id.json", 'w') as handle:
            json.dump(idx_to_ids, handle)
#             pickle.dump(idx_to_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
        div_cls_to_names = {lb:clsids_to_names[lb] for lb in div}
        div_clsids_to_idx = {lb:i+1 for i, lb in enumerate(div)}
        div_clsids_to_idx["background"] = 0
        div_cls_to_names["background"] = "background"
        
        with open(curr_save_dir+"/clsids_to_names.json", 'w') as handle:
            json.dump(div_cls_to_names, handle)
#             pickle.dump(div_cls_to_names, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(curr_save_dir+"/clsids_to_idx.json", 'w') as handle:
            json.dump(div_clsids_to_idx, handle)
#                 pickle.dump(div_clsids_to_idx, handle, protocol=pickle.HIGHEST_PROTOCOL)
                
        f = open(curr_save_dir+"/classes.txt", "w")
        f.write(str(len(div_cls_to_names)))
        f.close()

In [ ]:
create_multiple_subsets(train_anno, all_divisions, save_dir, "train", clsids_to_names)

In [ ]:
create_multiple_subsets(valid_anno, all_divisions, save_dir, "valid", clsids_to_names)

## Other Scripts

### run in console

```cmd
python scripts/02_data_analysis.py --anno-json-dir data_info/all/train/annotations/train-anno.json --idx-to-id-dir data_info/all/train/annotations/train-idx_to_id.json --clsids-to-idx-dir data_info/all/clsids_to_idx.json --save-dir data_info/all --clsids-to-names-dir data_info/all/clsids_to_names.json

```

```
python scripts/02_data_analysis.py --anno-json-dir data_info/subsets/0/annotations/train-anno.json --idx-to-id-dir data_info/subsets/0/annotations/train-idx_to_id.json --clsids-to-idx-dir data_info/subsets/0/clsids_to_idx.json --save-dir data_info/subsets/0 --clsids-to-names-dir data_info/subsets/0/clsids_to_names.json

python scripts/02_data_analysis.py --anno-json-dir data_info/subsets/1/annotations/train-anno.json --idx-to-id-dir data_info/subsets/1/annotations/train-idx_to_id.json --clsids-to-idx-dir data_info/subsets/1/clsids_to_idx.json --save-dir data_info/subsets/1 --clsids-to-names-dir data_info/subsets/1/clsids_to_names.json

python scripts/02_data_analysis.py --anno-json-dir data_info/subsets/2/annotations/train-anno.json --idx-to-id-dir data_info/subsets/2/annotations/train-idx_to_id.json --clsids-to-idx-dir data_info/subsets/2/clsids_to_idx.json --save-dir data_info/subsets/2 --clsids-to-names-dir data_info/subsets/2/clsids_to_names.json

python scripts/02_data_analysis.py --anno-json-dir data_info/subsets/3/annotations/train-anno.json --idx-to-id-dir data_info/subsets/3/annotations/train-idx_to_id.json --clsids-to-idx-dir data_info/subsets/3/clsids_to_idx.json --save-dir data_info/subsets/3 --clsids-to-names-dir data_info/subsets/3/clsids_to_names.json

python scripts/02_data_analysis.py --anno-json-dir data_info/subsets/4/annotations/train-anno.json --idx-to-id-dir data_info/subsets/4/annotations/train-idx_to_id.json --clsids-to-idx-dir data_info/subsets/4/clsids_to_idx.json --save-dir data_info/subsets/4 --clsids-to-names-dir data_info/subsets/4/clsids_to_names.json

python scripts/02_data_analysis.py --anno-json-dir data_info/subsets/5/annotations/train-anno.json --idx-to-id-dir data_info/subsets/5/annotations/train-idx_to_id.json --clsids-to-idx-dir data_info/subsets/5/clsids_to_idx.json --save-dir data_info/subsets/5 --clsids-to-names-dir data_info/subsets/5/clsids_to_names.json
```